# Machine Learning With Spark ML
In this lab assignment, you will complete a project by going through the following steps:
1. Get the data.
2. Discover the data to gain insights.
3. Prepare the data for Machine Learning algorithms.
4. Select a model and train it.
5. Fine-tune your model.
6. Present your solution.

As a dataset, we use the California Housing Prices dataset from the StatLib repository. This dataset was based on data from the 1990 California census. The dataset has the following columns
1. `longitude`: a measure of how far west a house is (a higher value is farther west)
2. `latitude`: a measure of how far north a house is (a higher value is farther north)
3. `housing_,median_age`: median age of a house within a block (a lower number is a newer building)
4. `total_rooms`: total number of rooms within a block
5. `total_bedrooms`: total number of bedrooms within a block
6. `population`: total number of people residing within a block
7. `households`: total number of households, a group of people residing within a home unit, for a block
8. `median_income`: median income for households within a block of houses
9. `median_house_value`: median house value for households within a block
10. `ocean_proximity`: location of the house w.r.t ocean/sea

---
# 1. Get the data
Let's start the lab by loading the dataset. The can find the dataset at `data/housing.csv`. To infer column types automatically, when you are reading the file, you need to set `inferSchema` to true. Moreover enable the `header` option to read the columns' name from the file.

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("lab1") \
    .getOrCreate()
housing = spark.read.options(inferSchema=True,header=True).csv('data/housing.csv')
housing.show()

/Users/mataymayrany/Library/Python/2.7/lib/python/site-packages/pyspark/context.py:227: DeprecationWarning: Support for Python 2 and Python 3 prior to version 3.6 is deprecated as of Spark 3.0. See also the plan for dropping Python 2 support at https://spark.apache.org/news/plan-for-dropping-python-2-support.html.
  DeprecationWarning)


+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

---
# 2. Discover the data to gain insights
Now it is time to take a look at the data. In this step we are going to take a look at the data a few different ways:
* See the schema and dimension of the dataset
* Look at the data itself
* Statistical summary of the attributes
* Breakdown of the data by the categorical attribute variable
* Find the correlation among different attributes
* Make new attributes by combining existing attributes

## 2.1. Schema and dimension
Print the schema of the dataset

In [2]:
housing.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



Print the number of records in the dataset.

In [3]:
housing.count()

20640

## 2.2. Look at the data
Print the first five records of the dataset.

In [4]:
housing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

Print the number of records with population more than 10000.

In [5]:
housing.filter(housing.population > 10000).count()

23

## 2.3. Statistical summary
Print a summary of the table statistics for the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. You can use the `describe` command.

In [6]:
housing.describe(['housing_median_age', 'total_rooms', 'median_house_value', 'population']).show()

+-------+------------------+------------------+------------------+------------------+
|summary|housing_median_age|       total_rooms|median_house_value|        population|
+-------+------------------+------------------+------------------+------------------+
|  count|             20640|             20640|             20640|             20640|
|   mean|28.639486434108527|2635.7630813953488|206855.81690891474|1425.4767441860465|
| stddev| 12.58555761211163|2181.6152515827944|115395.61587441359|  1132.46212176534|
|    min|               1.0|               2.0|           14999.0|               3.0|
|    max|              52.0|           39320.0|          500001.0|           35682.0|
+-------+------------------+------------------+------------------+------------------+



Print the maximum age (`housing_median_age`), the minimum number of rooms (`total_rooms`), and the average of house values (`median_house_value`).

In [7]:
from pyspark.sql.functions import max, min, mean, desc, count, col

housing.agg(max("housing_median_age"), min("total_rooms"), mean("median_house_value")).show()

+-----------------------+----------------+-----------------------+
|max(housing_median_age)|min(total_rooms)|avg(median_house_value)|
+-----------------------+----------------+-----------------------+
|                   52.0|             2.0|     206855.81690891474|
+-----------------------+----------------+-----------------------+



## 2.4. Breakdown the data by categorical data
Print the number of houses in different areas (`ocean_proximity`), and sort them in descending order.

In [8]:
housing.groupBy("ocean_proximity").agg(count("*")).sort(desc("count(1)")).show()

+---------------+--------+
|ocean_proximity|count(1)|
+---------------+--------+
|      <1H OCEAN|    9136|
|         INLAND|    6551|
|     NEAR OCEAN|    2658|
|       NEAR BAY|    2290|
|         ISLAND|       5|
+---------------+--------+



Print the average value of the houses (`median_house_value`) in different areas (`ocean_proximity`), and call the new column `avg_value` when print it.

In [9]:
housing.groupBy("ocean_proximity").agg(mean("median_house_value").alias("avg_value")).show()

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



Rewrite the above question in SQL.

In [10]:
housing.createOrReplaceTempView("df")
spark.sql("SELECT ocean_proximity, avg(median_house_value) AS avg_value from df GROUP BY ocean_proximity").show()

+---------------+------------------+
|ocean_proximity|         avg_value|
+---------------+------------------+
|         ISLAND|          380440.0|
|     NEAR OCEAN|249433.97742663656|
|       NEAR BAY|259212.31179039303|
|      <1H OCEAN|240084.28546409807|
|         INLAND|124805.39200122119|
+---------------+------------------+



## 2.5. Correlation among attributes
Print the correlation among the attributes `housing_median_age`, `total_rooms`, `median_house_value`, and `population`. To do so, first you need to put these attributes into one vector. Then, compute the standard correlation coefficient (Pearson) between every pair of attributes in this new vector. To make a vector of these attributes, you can use the `VectorAssembler` Transformer.

In [11]:
from pyspark.ml.feature import VectorAssembler

va = VectorAssembler(inputCols=["housing_median_age", "total_rooms", "median_house_value", "population"], outputCol="features")

housingAttrs = va.transform(housing).select("features")

housingAttrs.show(5)

+--------------------+
|            features|
+--------------------+
|[41.0,880.0,45260...|
|[21.0,7099.0,3585...|
|[52.0,1467.0,3521...|
|[52.0,1274.0,3413...|
|[52.0,1627.0,3422...|
+--------------------+
only showing top 5 rows



In [12]:
from pyspark.ml.linalg import Matrix

from pyspark.ml.stat import Correlation

from pyspark.sql import Row

matrix = Correlation.corr(housingAttrs, "features", "pearson").collect()[0][0]
print(matrix)

DenseMatrix([[ 1.        , -0.3612622 ,  0.10562341, -0.29624424],
             [-0.3612622 ,  1.        ,  0.13415311,  0.85712597],
             [ 0.10562341,  0.13415311,  1.        , -0.02464968],
             [-0.29624424,  0.85712597, -0.02464968,  1.        ]])


## 2.6. Combine and make new attributes
Now, let's try out various attribute combinations. In the given dataset, the total number of rooms in a block is not very useful, if we don't know how many households there are. What we really want is the number of rooms per household. Similarly, the total number of bedrooms by itself is not very useful, and we want to compare it to the number of rooms. And the population per household seems like also an interesting attribute combination to look at. To do so, add the three new columns to the dataset as below. We will call the new dataset the `housingExtra`.
```
rooms_per_household = total_rooms / households
bedrooms_per_room = bedrooms_per_room / total_rooms
population_per_household = population / households
```

In [13]:

housingCol1 = housing.withColumn('rooms_per_household', col("total_rooms") / col("households"))
housingCol2 = housingCol1.withColumn('bedrooms_per_room', col("total_bedrooms") / col("total_rooms"))
housingExtra = housingCol2.withColumn('population_per_household', col("population") / col("households"))

housingExtra.select("rooms_per_household", "bedrooms_per_room", "population_per_household").show(5)

+-------------------+-------------------+------------------------+
|rooms_per_household|  bedrooms_per_room|population_per_household|
+-------------------+-------------------+------------------------+
|  6.984126984126984|0.14659090909090908|      2.5555555555555554|
|  6.238137082601054|0.15579659106916466|       2.109841827768014|
|  8.288135593220339|0.12951601908657123|      2.8022598870056497|
| 5.8173515981735155|0.18445839874411302|       2.547945205479452|
|  6.281853281853282| 0.1720958819913952|      2.1814671814671813|
+-------------------+-------------------+------------------------+
only showing top 5 rows



---
## 3. Prepare the data for Machine Learning algorithms
Before going through the Machine Learning steps, let's first rename the label column from `median_house_value` to `label`.

In [14]:

renamedHousing = housingExtra.withColumnRenamed('median_house_value', 'label')
renamedHousing.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|358500.0|       NEAR BAY|  6.238137082601054|0.15579659106916466|       2.109841827768014|
|  -122.24|   37.85|              5

Now, we want to separate the numerical attributes from the categorical attribute (`ocean_proximity`) and keep their column names in two different lists. Moreover, sice we don't want to apply the same transformations to the predictors (features) and the label, we should remove the label attribute from the list of predictors. 

In [15]:
colLabel = "label"

colCat = "ocean_proximity"

renamedHousing.show()

colNum = renamedHousing.columns
colNum.remove(colLabel)
colNum.remove(colCat)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|358500.0|       NEAR BAY|  6.238137082601054|0.15579659106916466|       2.109841827768014|
|  -122.24|   37.85|              5

## 3.1. Prepare continuse attributes
### Data cleaning
Most Machine Learning algorithms cannot work with missing features, so we should take care of them. As a first step, let's find the columns with missing values in the numerical attributes. To do so, we can print the number of missing values of each continues attributes, listed in `colNum`.

In [16]:
from pyspark.sql.functions import isnan
for c in colNum: 
    print(c, renamedHousing.where(col(c).isNull() | isnan(c)).count())

('longitude', 0)
('latitude', 0)
('housing_median_age', 0)
('total_rooms', 0)
('total_bedrooms', 207)
('population', 0)
('households', 0)
('median_income', 0)
('rooms_per_household', 0)
('bedrooms_per_room', 207)
('population_per_household', 0)


As we observerd above, the `total_bedrooms` and `bedrooms_per_room` attributes have some missing values. One way to take care of missing values is to use the `Imputer` Transformer, which completes missing values in a dataset, either using the mean or the median of the columns in which the missing values are located. To use it, you need to create an `Imputer` instance, specifying that you want to replace each attribute's missing values with the "median" of that attribute.

In [17]:
from pyspark.ml.feature import Imputer

imputer = Imputer().setStrategy("median").setInputCols(["total_bedrooms", "bedrooms_per_room"]).setOutputCols(["total_bedrooms","bedrooms_per_room"])                                
imputedHousing = imputer.fit(renamedHousing).transform(renamedHousing)
    
imputedHousing.select("total_bedrooms", "bedrooms_per_room").show(5)

+--------------+-------------------+
|total_bedrooms|  bedrooms_per_room|
+--------------+-------------------+
|         129.0|0.14659090909090908|
|        1106.0|0.15579659106916466|
|         190.0|0.12951601908657123|
|         235.0|0.18445839874411302|
|         280.0| 0.1720958819913952|
+--------------+-------------------+
only showing top 5 rows



### Scaling
One of the most important transformations you need to apply to your data is feature scaling. With few exceptions, Machine Learning algorithms don't perform well when the input numerical attributes have very different scales. This is the case for the housing data: the total number of rooms ranges from about 6 to 39,320, while the median incomes only range from 0 to 15. Note that scaling the label attribues is generally not required.

One way to get all attributes to have the same scale is to use standardization. In standardization, for each value, first it subtracts the mean value (so standardized values always have a zero mean), and then it divides by the variance so that the resulting distribution has unit variance. To do this, we can use the `StandardScaler` Estimator. To use `StandardScaler`, again we need to convert all the numerical attributes into a big vectore of features using `VectorAssembler`, and then call `StandardScaler` on that vactor.

In [18]:
from pyspark.ml.feature import StandardScaler

va = VectorAssembler(inputCols = colNum, outputCol="features")
featuredHousing = va.transform(imputedHousing)

scaler = StandardScaler(withMean = True, withStd = True).setInputCol("features").setOutputCol("scaled")
scaledHousing = scaler.fit(featuredHousing).transform(featuredHousing)

scaledHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|            features|              scaled|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.5555555555555554|[-122.23,37.88,41...|[-1.3278030546902...|
|  -122.22|   37.86|              21.0|     7099.0|        1

## 3.2. Prepare categorical attributes
After imputing and scaling the continuse attributes, we should take care of the categorical attributes. Let's first print the number of distict values of the categirical attribute `ocean_proximity`.

In [19]:

renamedHousing.select(colCat).distinct().count()

5

### String indexer
Most Machine Learning algorithms prefer to work with numbers. So let's convert the categorical attribute `ocean_proximity` to numbers. To do so, we can use the `StringIndexer` that encodes a string column of labels to a column of label indices. The indices are in [0, numLabels), ordered by label frequencies, so the most frequent label gets index 0.

In [20]:

from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="ocean_proximity", outputCol="ocean_proximity_index")
idxHousing = indexer.fit(renamedHousing).transform(renamedHousing)

idxHousing.select("ocean_proximity_index").show(5)

+---------------------+
|ocean_proximity_index|
+---------------------+
|                  3.0|
|                  3.0|
|                  3.0|
|                  3.0|
|                  3.0|
+---------------------+
only showing top 5 rows



Now we can use this numerical data in any Machine Learning algorithm. You can look at the mapping that this encoder has learned using the `labels` method: "<1H OCEAN" is mapped to 0, "INLAND" is mapped to 1, etc.

In [21]:
indexer.fit(renamedHousing).labels

[u'<1H OCEAN', u'INLAND', u'NEAR OCEAN', u'NEAR BAY', u'ISLAND']

### One-hot encoding
Now, convert the label indices built in the last step into one-hot vectors. To do this, you can take advantage of the `OneHotEncoderEstimator` Estimator.

In [22]:
from pyspark.ml.feature import OneHotEncoder

encoder = OneHotEncoder(inputCol="ocean_proximity_index", outputCol="one_hot")
ohHousing = encoder.fit(idxHousing).transform(idxHousing)

ohHousing.select("one_hot").show(5)

+-------------+
|      one_hot|
+-------------+
|(4,[3],[1.0])|
|(4,[3],[1.0])|
|(4,[3],[1.0])|
|(4,[3],[1.0])|
|(4,[3],[1.0])|
+-------------+
only showing top 5 rows



---
# 4. Pipeline
As you can see, there are many data transformation steps that need to be executed in the right order. For example, you called the `Imputer`, `VectorAssembler`, and `StandardScaler` from left to right. However, we can use the `Pipeline` class to define a sequence of Transformers/Estimators, and run them in order. A `Pipeline` is an `Estimator`, thus, after a Pipeline's `fit()` method runs, it produces a `PipelineModel`, which is a `Transformer`.

Now, let's create a pipeline called `numPipeline` to call the numerical transformers you built above (`imputer`, `va`, and `scaler`) in the right order from left to right, as well as a pipeline called `catPipeline` to call the categorical transformers (`indexer` and `encoder`). Then, put these two pipelines `numPipeline` and `catPipeline` into one pipeline.

In [23]:
from pyspark.ml import Pipeline, PipelineModel

numPipeline = Pipeline(stages = [imputer, va, scaler])
catPipeline = Pipeline(stages = [indexer, encoder])
pipeline = Pipeline(stages = [numPipeline, catPipeline])
newHousing = pipeline.fit(renamedHousing).transform(renamedHousing)

newHousing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+---------------------+-------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|   label|ocean_proximity|rooms_per_household|  bedrooms_per_room|population_per_household|            features|              scaled|ocean_proximity_index|      one_hot|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+--------+---------------+-------------------+-------------------+------------------------+--------------------+--------------------+---------------------+-------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|452600.0|       NEAR BAY|  6.984126984126984|0.14659090909090908|      2.555555555555

Now, use `VectorAssembler` to put all attributes of the final dataset `newHousing` into a big vector, and call the new column `features`.

In [24]:
newerHousing = newHousing.drop("features")
va2 = VectorAssembler(inputCols = ["scaled", "one_hot"], outputCol = "features")
dataset = va2.transform(newerHousing).select("features", "label")

dataset.show(5)

+--------------------+--------+
|            features|   label|
+--------------------+--------+
|[-1.3278030546902...|452600.0|
|[-1.3228118684350...|358500.0|
|[-1.3327942409452...|352100.0|
|[-1.3377854272003...|341300.0|
|[-1.3377854272003...|342200.0|
+--------------------+--------+
only showing top 5 rows



---
# 5. Make a model
Here we going to make four different regression models:
* Linear regression model
* Decission tree regression
* Random forest regression
* Gradient-booster forest regression

But, before giving the data to train a Machine Learning model, let's first split the data into training dataset (`trainSet`) with 80% of the whole data, and test dataset (`testSet`) with 20% of it.

In [25]:

[trainSet, testSet] = dataset.randomSplit([0.8, 0.2])
print(trainSet.count(), testSet.count())

(16501, 4139)


## 5.1. Linear regression model
Now, train a Linear Regression model using the `LinearRegression` class. Then, print the coefficients and intercept of the model, as well as the summary of the model over the training set by calling the `summary` method.

In [26]:
from pyspark.ml.regression import LinearRegression
#Train Model
lr = LinearRegression()
lrModel = lr.fit(trainSet)
trainingSummary = lrModel.summary

print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)

Coefficients: [-55760.9282739,-56348.5259596,14277.5569958,4365.17100387,2722.6613519,-45471.6677255,44009.1519047,77188.0305423,8640.49363,15863.4328384,3029.51244626,-118842.715288,-153521.352621,-113919.003942,-122717.417358]
Intercept: 336732.448043
RMSE: 68276.404536


Now, use `RegressionEvaluator` to measure the root-mean-square-erroe (RMSE) of the model on the test dataset.

In [27]:
from pyspark.ml.evaluation import RegressionEvaluator

# make predictions on the test data
predictions = lrModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

# select (prediction, true label) and compute test error.
evaluator = RegressionEvaluator(metricName = "rmse", predictionCol = "prediction", labelCol = "label")
rmse = evaluator.evaluate(predictions)
print("RMSE: %f" % rmse)


+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
| 204710.4797380651| 94600.0|[-2.3859345407710...|
|184007.09587175286|111400.0|[-2.3410138644752...|
| 145018.7711922204| 58100.0|[-2.3160579331997...|
|121765.41470395797| 68400.0|[-2.3160579331997...|
|146653.33784345852| 70000.0|[-2.3060755606895...|
+------------------+--------+--------------------+
only showing top 5 rows

RMSE: 66612.305107


## 5.2. Decision tree regression
Repeat what you have done on Regression Model to build a Decision Tree model. Use the `DecisionTreeRegressor` to make a model and then measure its RMSE on the test dataset.

In [28]:

from pyspark.ml.regression import DecisionTreeRegressor 
from pyspark.ml.evaluation import RegressionEvaluator

dt = DecisionTreeRegressor()

# train the model
dtModel = dt.fit(trainSet)

# make predictions on the test data
predictions = dtModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

# select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(metricName = "rmse", predictionCol = "prediction", labelCol = "label")
rmse = evaluator.evaluate(predictions)
print("RMSE: %f" % rmse)

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|172051.08210890234| 94600.0|[-2.3859345407710...|
|151203.94736842104|111400.0|[-2.3410138644752...|
|151203.94736842104| 58100.0|[-2.3160579331997...|
|172051.08210890234| 68400.0|[-2.3160579331997...|
|151203.94736842104| 70000.0|[-2.3060755606895...|
+------------------+--------+--------------------+
only showing top 5 rows

RMSE: 66056.675465


## 5.3. Random forest regression
Let's try the test error on a Random Forest Model. Youcan use the `RandomForestRegressor` to make a Random Forest model.

In [29]:

from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

rf = RandomForestRegressor()

# train the model
rfModel = rf.fit(trainSet)

# make predictions on the test data
predictions = rfModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

# select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(metricName = "rmse", predictionCol = "prediction", labelCol = "label")
rmse = evaluator.evaluate(predictions)
print("RMSE: %f" % rmse)

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
| 179891.1172160705| 94600.0|[-2.3859345407710...|
| 175176.6541302617|111400.0|[-2.3410138644752...|
|173963.24881649765| 58100.0|[-2.3160579331997...|
|166071.44132924877| 68400.0|[-2.3160579331997...|
|  166490.054922364| 70000.0|[-2.3060755606895...|
+------------------+--------+--------------------+
only showing top 5 rows

RMSE: 64554.423266


## 5.4. Gradient-boosted tree regression
Fianlly, we want to build a Gradient-boosted Tree Regression model and test the RMSE of the test data. Use the `GBTRegressor` to build the model.

In [30]:

from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

gb = GBTRegressor()

# train the model
gbModel = gb.fit(trainSet)

# make predictions on the test data
predictions = gbModel.transform(testSet)
predictions.select("prediction", "label", "features").show(5)

# select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(metricName = "rmse", predictionCol = "prediction", labelCol = "label")
rmse = evaluator.evaluate(predictions)
print("RMSE: %f" % rmse)

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|129636.40288275055| 94600.0|[-2.3859345407710...|
|104557.39549526614|111400.0|[-2.3410138644752...|
| 85969.45339746148| 58100.0|[-2.3160579331997...|
|122145.37819005008| 68400.0|[-2.3160579331997...|
| 74464.93659001043| 70000.0|[-2.3060755606895...|
+------------------+--------+--------------------+
only showing top 5 rows

RMSE: 53547.104731


---
# 6. Hyperparameter tuning
An important task in Machie Learning is model selection, or using data to find the best model or parameters for a given task. This is also called tuning. Tuning may be done for individual Estimators such as LinearRegression, or for entire Pipelines which include multiple algorithms, featurization, and other steps. Users can tune an entire Pipeline at once, rather than tuning each element in the Pipeline separately. MLlib supports model selection tools, such as `CrossValidator`. These tools require the following items:
* Estimator: algorithm or Pipeline to tune (`setEstimator`)
* Set of ParamMaps: parameters to choose from, sometimes called a "parameter grid" to search over (`setEstimatorParamMaps`)
* Evaluator: metric to measure how well a fitted Model does on held-out test data (`setEvaluator`)

`CrossValidator` begins by splitting the dataset into a set of folds, which are used as separate training and test datasets. For example with `k=3` folds, `CrossValidator` will generate 3 (training, test) dataset pairs, each of which uses 2/3 of the data for training and 1/3 for testing. To evaluate a particular `ParamMap`, `CrossValidator` computes the average evaluation metric for the 3 Models produced by fitting the Estimator on the 3 different (training, test) dataset pairs. After identifying the best `ParamMap`, `CrossValidator` finally re-fits the Estimator using the best ParamMap and the entire dataset.

Below, use the `CrossValidator` to select the best Random Forest model. To do so, you need to define a grid of parameters. Let's say we want to do the search among the different number of trees (1, 5, and 10), and different tree depth (5, 10, and 15).

In [31]:

from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator

paramGrid = ParamGridBuilder().addGrid(rfModel.maxDepth, [5, 10, 15]).addGrid(rfModel.numTrees, [1, 5, 10]).build()

evaluator = RegressionEvaluator(metricName = "rmse", predictionCol = "prediction", labelCol = "label")
# rfPipeline = Pipeline(stages = [rfModel])
cv = CrossValidator(estimator = rf, estimatorParamMaps = paramGrid, evaluator = evaluator)
cvModel = cv.fit(dataset)

predictions = cvModel.transform(dataset)
predictions.select("prediction", "label", "features").show(5)

rmse = evaluator.evaluate(predictions)
print("RMSE: %f" % rmse)

+------------------+--------+--------------------+
|        prediction|   label|            features|
+------------------+--------+--------------------+
|          444183.7|452600.0|[-1.3278030546902...|
|378053.43333333335|358500.0|[-1.3228118684350...|
| 394571.1666666666|352100.0|[-1.3327942409452...|
| 330696.1471177945|341300.0|[-1.3377854272003...|
|275710.14642857143|342200.0|[-1.3377854272003...|
+------------------+--------+--------------------+
only showing top 5 rows

RMSE: 30727.530915


---
# 7. An End-to-End Classification Test
As the last step, you are given a dataset called `data/ccdefault.csv`. The dataset represents default of credit card clients. It has 30,000 cases and 24 different attributes. More details about the dataset is available at `data/ccdefault.txt`. In this task you should make three models, compare their results and conclude the ideal solution. Here are the suggested steps:
1. Load the data.
2. Carry out some exploratory analyses (e.g., how various features and the target variable are distributed).
3. Train a model to predict the target variable (risk of `default`).
  - Employ three different models (logistic regression, decision tree, and random forest).
  - Compare the models' performances (e.g., AUC).
  - Defend your choice of best model (e.g., what are the strength and weaknesses of each of these models?).
4. What more would you do with this data? Anything to help you devise a better solution?

In [32]:

ccdefault = spark.read.options(inferSchema=True,header=True).csv('data/ccdefault.csv')
ccdefault.select("pay_0", "BILL_AMT2").show(5)

+-----+---------+
|pay_0|BILL_AMT2|
+-----+---------+
|    2|     3102|
|   -1|     1725|
|    0|    14027|
|    0|    48233|
|   -1|     5670|
+-----+---------+
only showing top 5 rows



In [33]:
ccdefault.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- LIMIT_BAL: integer (nullable = true)
 |-- SEX: integer (nullable = true)
 |-- EDUCATION: integer (nullable = true)
 |-- MARRIAGE: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- PAY_0: integer (nullable = true)
 |-- PAY_2: integer (nullable = true)
 |-- PAY_3: integer (nullable = true)
 |-- PAY_4: integer (nullable = true)
 |-- PAY_5: integer (nullable = true)
 |-- PAY_6: integer (nullable = true)
 |-- BILL_AMT1: integer (nullable = true)
 |-- BILL_AMT2: integer (nullable = true)
 |-- BILL_AMT3: integer (nullable = true)
 |-- BILL_AMT4: integer (nullable = true)
 |-- BILL_AMT5: integer (nullable = true)
 |-- BILL_AMT6: integer (nullable = true)
 |-- PAY_AMT1: integer (nullable = true)
 |-- PAY_AMT2: integer (nullable = true)
 |-- PAY_AMT3: integer (nullable = true)
 |-- PAY_AMT4: integer (nullable = true)
 |-- PAY_AMT5: integer (nullable = true)
 |-- PAY_AMT6: integer (nullable = true)
 |-- DEFAULT: integer (nullable = tru

In [34]:
ccdefault.describe(["LIMIT_BAL", "EDUCATION", "AGE", "PAY_0", "BILL_AMT1", "PAY_AMT1", "DEFAULT"]).show(5)

+-------+------------------+------------------+-----------------+------------------+-----------------+-----------------+-------------------+
|summary|         LIMIT_BAL|         EDUCATION|              AGE|             PAY_0|        BILL_AMT1|         PAY_AMT1|            DEFAULT|
+-------+------------------+------------------+-----------------+------------------+-----------------+-----------------+-------------------+
|  count|             30000|             30000|            30000|             30000|            30000|            30000|              30000|
|   mean|167484.32266666667|1.8531333333333333|          35.4855|           -0.0167|       51223.3309|        5663.5805|             0.2212|
| stddev|129747.66156720246|0.7903486597207269|9.217904068090155|1.1238015279973335|73635.86057552966|16563.28035402577|0.41506180569093254|
|    min|             10000|                 0|               21|                -2|          -165580|                0|                  0|
|    max|    

In [37]:
# see if any nulls exist
numberedColumns = ccdefault.columns
for c in numberedColumns: 
    print(c, ccdefault.where(col(c).isNull() | isnan(c)).count())

('ID', 0)
('LIMIT_BAL', 0)
('SEX', 0)
('EDUCATION', 0)
('MARRIAGE', 0)
('AGE', 0)
('PAY_0', 0)
('PAY_2', 0)
('PAY_3', 0)
('PAY_4', 0)
('PAY_5', 0)
('PAY_6', 0)
('BILL_AMT1', 0)
('BILL_AMT2', 0)
('BILL_AMT3', 0)
('BILL_AMT4', 0)
('BILL_AMT5', 0)
('BILL_AMT6', 0)
('PAY_AMT1', 0)
('PAY_AMT2', 0)
('PAY_AMT3', 0)
('PAY_AMT4', 0)
('PAY_AMT5', 0)
('PAY_AMT6', 0)
('DEFAULT', 0)


In [38]:
# drop id, seperate other columns into numeric, categorical, and label
renamedDefault = ccdefault.withColumnRenamed('DEFAULT', 'label')
labelColumn = 'label'
categoryColumns = ['SEX', 'EDUCATION', 'MARRIAGE']

numberedColumns = renamedDefault.columns
numberedColumns.remove('SEX')
numberedColumns.remove('EDUCATION')
numberedColumns.remove('MARRIAGE')
numberedColumns.remove(labelColumn)


In [39]:
# combine numeric features into vectors
va = VectorAssembler(inputCols = numberedColumns, outputCol = "features")
featuredDefault = va.transform(renamedDefault)
featuredDefault.select("features").show(5)

+--------------------+
|            features|
+--------------------+
|[1.0,20000.0,24.0...|
|[2.0,120000.0,26....|
|[3.0,90000.0,34.0...|
|[4.0,50000.0,37.0...|
|[5.0,50000.0,57.0...|
+--------------------+
only showing top 5 rows



In [40]:
# scale numberic features
scaler = StandardScaler(withMean = True, withStd = True).setInputCol("features").setOutputCol("scaled")
scaledDefault = scaler.fit(featuredDefault).transform(featuredDefault)

scaledDefault.show(5)

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+--------------------+--------------------+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|label|            features|              scaled|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+--------------------+--------------------+
|  1|    20000|  2|        2|       1| 24|    2|    2|   -1|   -1|   -2|   -2|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|    1|[1.0,20000.0,24.0...|[-1.7319642067123...|
|  2|   120000|  2|        2|       

In [41]:
# encode categorical features to one hot vectors
encoder = OneHotEncoder(inputCols = categoryColumns, outputCols=["one_hot_sex", "one_hot_education", "one_hot_marriage"])
ohDefault = encoder.fit(scaledDefault).transform(scaledDefault)

ohDefault.select("one_hot_sex", "one_hot_education", "one_hot_marriage").show(5)

+-------------+-----------------+----------------+
|  one_hot_sex|one_hot_education|one_hot_marriage|
+-------------+-----------------+----------------+
|    (2,[],[])|    (6,[2],[1.0])|   (3,[1],[1.0])|
|    (2,[],[])|    (6,[2],[1.0])|   (3,[2],[1.0])|
|    (2,[],[])|    (6,[2],[1.0])|   (3,[2],[1.0])|
|    (2,[],[])|    (6,[2],[1.0])|   (3,[1],[1.0])|
|(2,[1],[1.0])|    (6,[2],[1.0])|   (3,[1],[1.0])|
+-------------+-----------------+----------------+
only showing top 5 rows



In [42]:
# combine features vectors from numeric attributes with one hot encoded vectors from categorical attributes
droppedFeaturesDefualt = ohDefault.drop("features")
va2 = VectorAssembler(inputCols = ["scaled", "one_hot_sex", "one_hot_education", "one_hot_marriage"], outputCol = "features")
finalDataset = va2.transform(droppedFeaturesDefualt).select("label", "features")
finalDataset.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    1|[-1.7319642067123...|
|    1|[-1.7318487385830...|
|    0|[-1.7317332704536...|
|    0|[-1.7316178023242...|
|    0|[-1.7315023341948...|
+-----+--------------------+
only showing top 5 rows



In [43]:
[trainSet, testSet] = finalDataset.randomSplit([0.8, 0.2])
print(trainSet.count(), testSet.count())

(24036, 5964)


In [44]:
#Logistic regression model
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator


lr = LogisticRegression()

# train the model
lrModel = lr.fit(trainSet)

# make predictions on the test data
predictions = lrModel.transform(testSet)
predictions.show(5)


# select (prediction, true label) and compute test error
evaluator = BinaryClassificationEvaluator()
evaluation = evaluator.evaluate(predictions)

print("evaluation (area under ROC): %f" % evaluation)


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|[-1.7313868660654...|[1.24377133255332...|[0.77621978700157...|       0.0|
|    0|[-1.7312713979360...|[2.55353046119983...|[0.92781033642653...|       0.0|
|    0|[-1.7302321847716...|[0.64850202317081...|[0.65667281766819...|       0.0|
|    0|[-1.7300012485128...|[2.72955985920687...|[0.93874853409275...|       0.0|
|    0|[-1.7280382903132...|[2.99907752238951...|[0.95253243496235...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

evaluation (area under ROC): 0.725232


In [46]:
from pyspark.ml.classification import DecisionTreeClassifier
# Decision Tree model
dt = DecisionTreeClassifier()

# train the model
dtModel = dt.fit(trainSet)

# make predictions on the test data
predictions = dtModel.transform(testSet)
predictions.show(5)

# select (prediction, true label) and compute test error
evaluator = BinaryClassificationEvaluator()
evaluation = evaluator.evaluate(predictions)

print("evaluation (area under ROC): %f" % evaluation)

+-----+--------------------+----------------+--------------------+----------+
|label|            features|   rawPrediction|         probability|prediction|
+-----+--------------------+----------------+--------------------+----------+
|    0|[-1.7313868660654...|[16902.0,2808.0]|[0.85753424657534...|       0.0|
|    0|[-1.7312713979360...|[16902.0,2808.0]|[0.85753424657534...|       0.0|
|    0|[-1.7302321847716...|   [779.0,453.0]|[0.63230519480519...|       0.0|
|    0|[-1.7300012485128...|[16902.0,2808.0]|[0.85753424657534...|       0.0|
|    0|[-1.7280382903132...|[16902.0,2808.0]|[0.85753424657534...|       0.0|
+-----+--------------------+----------------+--------------------+----------+
only showing top 5 rows

evaluation (area under ROC): 0.322248


In [48]:
from pyspark.ml.classification import RandomForestClassifier

# Random forest model
rf = RandomForestClassifier()

# train the model
rfModel = rf.fit(trainSet)

# make predictions on the test data
predictions = rfModel.transform(testSet)
predictions.show(5)

# select (prediction, true label) and compute test error
evaluator = BinaryClassificationEvaluator()
evaluation = evaluator.evaluate(predictions)

print("evaluation (area under ROC): %f" % evaluation)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|[-1.7313868660654...|[16.7442552149595...|[0.83721276074797...|       0.0|
|    0|[-1.7312713979360...|[17.4950678685026...|[0.87475339342513...|       0.0|
|    0|[-1.7302321847716...|[14.2774732452830...|[0.71387366226415...|       0.0|
|    0|[-1.7300012485128...|[17.4950678685026...|[0.87475339342513...|       0.0|
|    0|[-1.7280382903132...|[17.0390296809847...|[0.85195148404923...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

evaluation (area under ROC): 0.771700
